In [1]:
from tensorflow import keras

In [ ]:
model_A = keras.models.load_model("my_model_A.h5")  # load a preexisting model
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])  # assign all layers except the output layer. Notice that the layers are NOT copied
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))  # add a new output layer

In [ ]:
# to avoid binding two models, clone them
model_A_clone = keras.models.clone_model(model_A)  # clone the architecture
model_A_clone.set_weights(model_A.get_weights())  # copy the weights

In [ ]:
# freeze the copied layers. The model must always be compiled after any layers are froze/unfroze
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer="sgd",
                     metrics=["accuracy"])

In [ ]:
# train for a couple of epochs, unfreeze, and train some more with a smaller learning rate
history = model_B_on_A.fit(X_train_B, y_train_B,
                           epochs=4,
                           validation_data=(X_valid_B, y_valid_B)
                           )
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

optimizer = keras.optimizers.SGD(learning_rate=1e-4)
model_B_on_A.compile(loss='binary_crossentropy',
                     optimizer=optimizer,
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B,
                           epochs=16,
                           validation_data=(X_valid_B, y_valid_B))


In [ ]:
model_B_on_A.evaluate(X_test_B, y_test_B)